# DITP experimentation n°2 - part 2
# Notebook for LM campaign on the 19th september that begins on the 26th of september
Analysis deadlines: 10 october, 10 november, 31 december

The files are to received from Link Mobility, 
We need to fill in the type beneficiairy/ object received/ mail received in the final DITP exported consolidated file

# Mapping table
| Type de bénéficiaire | Objet reçu | Mail reçu |
|----------------------|------------|-----------|
| 1 (parent)           | 0 (contrôle)| 0 (contrôle) |
| 1 (parent)           | 0 (contrôle)| 1 (test)      |
| 1 (parent)           | 1 (test)    | 0 (contrôle) |
| 1 (parent)           | 1 (test)    | 1 (test)      |
| 2 (Ben direct)       | 0 (contrôle)| 0 (contrôle) |
| 2 (Ben direct)       | 0 (contrôle)| 1 (test)      |
| 2 (Ben direct)       | 1 (test)    | 0 (contrôle) |
| 2 (Ben direct)       | 1 (test)    | 1 (test)      |


In [ ]:
from dotenv import load_dotenv
import os
import numpy as np

load_dotenv()

activated_pass_sport_path_file = os.environ['ACTIVATED_PASS_SPORT_PATH_FILE']
consolidated_original_path_file = os.environ['CAMPAIGN_LINK_MOBILITY_19_SEPTEMBER_CONSOLIDATED_OUTPUT_PATHFILE']
ditp_analysis_output_path_file = os.environ['DITP_ANALYSIS_EXPORT_OUPUT_PATH_FILE']

# Columns for exported CSV to DITP
type_benef_col = 'type_benef' # 1=parent, 2=direct
obj_received_col = 'objet_recu' # 0=control, 1=test
mail_received_col = 'mail_recu' # 0=control, 1=test
activated_col = 'recours' # 0=non, 1=oui
benef_gender_col = 'beneficiaire_genre' # 1=Femme, 2=Homme
benef_age_col = 'beneficiaire_age'
parents_gender_col = 'parents_genre' # 1=Femme, 2=Homme
parents_age_col = 'parents_age'
postal_code_col = 'code_postal'
child_age_col = 'age_enfant'
qpv_col = 'qpv'
zrr_col ='zrr'
residential_area_col = 'zone_habitation'

# Additionnal columns for DITP
type_benef = {
  'parent': 1,
  'benef': 2
}

obj_received = {
  'control': 0,
  'test': 1
}

mail_received = {
  'control': 0,
  'test': 1
}

In [ ]:
import pandas as pd

# Load into dataframes the 8 files that were initially sent to Link Mobility
df_100 = pd.read_csv(os.environ['DITP_CAMPAIGN_FROM_26_SEPTEMBER_PART_100_PATHFILE'])
df_101 = pd.read_csv(os.environ['DITP_CAMPAIGN_FROM_26_SEPTEMBER_PART_101_PATHFILE'])
df_110 = pd.read_csv(os.environ['DITP_CAMPAIGN_FROM_26_SEPTEMBER_PART_110_PATHFILE'])
df_111 = pd.read_csv(os.environ['DITP_CAMPAIGN_FROM_26_SEPTEMBER_PART_111_PATHFILE'])
df_200 = pd.read_csv(os.environ['DITP_CAMPAIGN_FROM_26_SEPTEMBER_PART_200_PATHFILE'])
df_201 = pd.read_csv(os.environ['DITP_CAMPAIGN_FROM_26_SEPTEMBER_PART_201_PATHFILE'])
df_210 = pd.read_csv(os.environ['DITP_CAMPAIGN_FROM_26_SEPTEMBER_PART_210_PATHFILE'])
df_211 = pd.read_csv(os.environ['DITP_CAMPAIGN_FROM_26_SEPTEMBER_PART_211_PATHFILE'])

In [ ]:
# Initialise type_benef, object received & mail received for each of the 8 files initially sent to Link Mobility
df_100[[type_benef_col, obj_received_col, mail_received_col]] = [type_benef['parent'], obj_received['control'], mail_received['control']]
df_101[[type_benef_col, obj_received_col, mail_received_col]] = [type_benef['parent'], obj_received['control'], mail_received['test']]
df_110[[type_benef_col, obj_received_col, mail_received_col]] = [type_benef['parent'], obj_received['test'], mail_received['control']]
df_111[[type_benef_col, obj_received_col, mail_received_col]] = [type_benef['parent'], obj_received['test'], mail_received['test']]
df_200[[type_benef_col, obj_received_col, mail_received_col]] = [type_benef['benef'], obj_received['control'], mail_received['control']]
df_201[[type_benef_col, obj_received_col, mail_received_col]] = [type_benef['benef'], obj_received['control'], mail_received['test']]
df_210[[type_benef_col, obj_received_col, mail_received_col]] = [type_benef['benef'], obj_received['test'], mail_received['control']]
df_211[[type_benef_col, obj_received_col, mail_received_col]] = [type_benef['benef'], obj_received['test'], mail_received['test']]

In [ ]:
# Merge all the files that were sent to Link Mobility into one dataframe
merged_df_to_update = pd.concat([
  df_100,
  df_101,
  df_110,
  df_111,
  df_200,
  df_201,
  df_210,
  df_211
], ignore_index=True).reset_index()

In [ ]:
# Original file that contains exhaustive information about beneficiaire & allocataire
original_df = pd.read_csv(consolidated_original_path_file, dtype={
  'email': 'str',
  'allocataire_age': 'Int64',
  'allocataire_genre': 'str',
  'beneficiaire_age': 'Int64',
  'beneficiaire_genre': 'str',
  'allocataire_qualite': 'str',
  'code_postal': 'str',
  'zrr': 'boolean',
  'qpv': 'boolean'
})

In [ ]:
# Merge DITP with original CSV that was sent to Link Mobility for the campaign that contains additionnal information such as
# beneficiary age, allocataire age, zzr, qpv, 
merged_df_updated = pd.merge(merged_df_to_update, original_df, how='left', on=['code'], suffixes=(None,'_new'))

In [ ]:
merged_df_updated[['qpv', 'zrr']] = merged_df_updated[['qpv', 'zrr']].fillna(False)

In [ ]:
# Load csv file into dataframe that contains people who activated their pass Sport
df_people_who_activated = pd.read_csv(activated_pass_sport_path_file, dtype={
  'id': 'str',
  'id_psp': 'str',
})

df_people_who_activated = df_people_who_activated.rename(columns={'id': 'str', 'id_psp': 'code'})
df_people_who_activated[activated_col] = 1

In [ ]:
# Fill in people who have activated their pass sport
merged_df_updated = pd.merge(
  merged_df_updated, 
  df_people_who_activated[['code', activated_col]],
  how='left',
  on=['code'], 
  suffixes=(None, '_new')
)

In [ ]:
# Fill column dedicated to activated pass sport
merged_df_updated[activated_col] = merged_df_updated[activated_col].fillna(value=0)
merged_df_updated[activated_col] = merged_df_updated[activated_col].astype(int)

In [ ]:
# Map genders
merged_df_updated['allocataire_genre'] = np.where(merged_df_updated['allocataire_genre'] == 'F', 1, 2)
merged_df_updated['beneficiaire_genre'] = np.where(merged_df_updated['beneficiaire_genre'] == 'F', 1, 2)

In [ ]:
# Create <residential_area_col>, exclude qpv and zrr when both are true because it isn't reliable
merged_df_updated[residential_area_col] = np.where(
    merged_df_updated['qpv'] & merged_df_updated['zrr'],  # Condition: Both qpv and zrr are True
    '',  # If both are True, set empty string
    np.where(
      merged_df_updated['qpv'], 'qpv', 
      np.where(
        merged_df_updated['zrr'], 'zrr', ''
      )
    )  # Otherwise, set 'qpv', 'zrr', or empty string
)

In [ ]:
columns_to_keep = [
  type_benef_col, 
  obj_received_col, 
  mail_received_col, 
  activated_col,
  parents_age_col,
  parents_gender_col,
  benef_gender_col,
  benef_age_col,
  postal_code_col,
  residential_area_col
]

final_df = merged_df_updated.rename(columns={
  'allocataire_genre': parents_gender_col,
  'allocataire_age': parents_age_col,
  'beneficiaire_genre': benef_gender_col,
  'beneficiaire_age': benef_age_col
})

final_df = final_df[columns_to_keep]

In [ ]:
# Export final csv file for DITP
final_df.to_csv(ditp_analysis_output_path_file, index=False)